# 22-Feature-generation

In [14]:
# to get more relationship between whether the employeee will be success or not

In [15]:
'''
feature including

words frequency: sales, investment, data, 

number density

document length

positive emotion score
negative emotion score

two other different emotion scores


'''

'\nfeature including\n\nwords frequency: sales, investment, data, \n\nnumber density\n\ndocument length\n\npositive emotion score\nnegative emotion score\n\ntwo other different emotion scores\n\n\n'

In [16]:
import pandas as pd
import os
import json
from collections import Counter
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import torch
import readability

In [17]:
feature_data = pd.DataFrame()

In [18]:
file_path = []
file_name = []
home_path = '/home/guoy20/Data/json/'
label = []
classification = ['Good','Bad','Inter']
for c in classification:
    path1 = home_path + c 
    filename = os.listdir(path1)
    for name in filename:
        if name != '.ipynb_checkpoints':
            file_path.append(path1 + '/' + name)

            if c == 'Good':
                label.append(2)
            elif c == 'Inter':
                label.append(1)
            elif c == 'Bad':
                label.append(0)
            file_name.append(name)

In [19]:
def count_word(w,file_path):
    result = []
    for file in file_path:
        try:
            with open(file) as f:
                data = json.load(f)
        except:
            result.append(0)
            continue
        word_count = 0
        for item in data:

            for i in w:
                try:
                    word_count += list(item.values())[0].count(i)
                except:
                    word_count = word_count
        result.append(word_count)
    return result

In [20]:
sale_count = count_word(['sales','sale','Sales','Sale'],file_path = file_path)
financial_count = count_word(['financial','Financial'],file_path = file_path)
investment_count = count_word(['investment','Investment'],file_path = file_path)
advisor_count = count_word(['advisor','Advisor','advisors','Advisors'],file_path = file_path)

In [21]:
feature_data['label'] = label
feature_data['resume_name'] = file_name
feature_data['sales_count'] = sale_count
feature_data['financial_count'] = financial_count
feature_data['investment_count'] = investment_count
feature_data['advisor_count'] = advisor_count

In [22]:
feature_data

,label,resume_name,sales_count,financial_count,investment_count,advisor_count
0,2,"21. Bocinsky, James_Resume.json",4,9,1,11
1,2,82. Linhardt Evan_Resume.json,0,0,0,0
2,2,"85. Malot, Dylan_Resume .PDF.json",0,2,1,3
3,2,"109. Poole, Blake - Resume.json",16,2,5,0
4,2,"136. Underkofler, Jon_Resume.json",10,10,8,7
...,...,...,...,...,...,...
125,1,"132. Statton, Robert H_BD.json",29,4,2,0
126,1,"64. Kaempf, Andrea_Resume - FA - Seattle.json",18,0,7,3
127,1,"99. Moye, Jerry_Resume.json",64,0,0,0
128,1,"29. Corello, Daniel_Resume.json",63,6,1,6


In [23]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

/home/guoy20/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1112: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [24]:
def get_emotion(text):
    input_ids = tokenizer.encode(str(text) + '</s>', return_tensors='pt')
    output = model.generate(input_ids=input_ids,
               max_length=2)
    dec = [tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label
get_emotion("") # Output: 'joy'
 
get_emotion("i have a feeling i kinda lost my best friend")

2023-03-10 15:18:36.075997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 15:20:02.199367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 15:20:02.199449: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 15:21:53.470122: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

'<pad> sadness'

In [25]:
final = []
for file in file_path:
    try:
        with open(file) as f:
            data = json.load(f)
    except:
        final.append({'joy':0,'fear':0,'sadness':0,'anger':0,'love':0,'surprise':0})
        continue
    document = {'joy':0,'fear':0,'sadness':0,'anger':0,'love':0,'surprise':0}
    count = 0
    for item in data:
        count += 1
        try:
            text = list(item.values())[0]
        except:
            continue
            
        
        
        result = get_emotion(text)[6:]
        if result in list(document.keys()):
            document[result] += 1
    r = {}
    for key in document:
        r[key] = document[key]/count
    final.append(r)
    

In [26]:
joy = [i['joy'] for i in final]
anger = [i['anger'] for i in final]
fear = [i['fear'] for i in final]

In [27]:
feature_data['joy'] = joy
feature_data['anger'] = anger 
feature_data['fear'] = fear

In [28]:
feature_data

,label,resume_name,sales_count,financial_count,investment_count,advisor_count,joy,anger,fear
0,2,"21. Bocinsky, James_Resume.json",4,9,1,11,0.620690,0.172414,0.206897
1,2,82. Linhardt Evan_Resume.json,0,0,0,0,0.769231,0.076923,0.153846
2,2,"85. Malot, Dylan_Resume .PDF.json",0,2,1,3,0.769231,0.076923,0.153846
3,2,"109. Poole, Blake - Resume.json",16,2,5,0,0.782609,0.086957,0.108696
4,2,"136. Underkofler, Jon_Resume.json",10,10,8,7,0.780488,0.024390,0.195122
...,...,...,...,...,...,...,...,...,...
125,1,"132. Statton, Robert H_BD.json",29,4,2,0,0.661017,0.118644,0.135593
126,1,"64. Kaempf, Andrea_Resume - FA - Seattle.json",18,0,7,3,0.610169,0.186441,0.169492
127,1,"99. Moye, Jerry_Resume.json",64,0,0,0,0.594203,0.202899,0.202899
128,1,"29. Corello, Daniel_Resume.json",63,6,1,6,0.604651,0.093023,0.302326


In [29]:
number_count = []
for file in file_path:
    try:
        with open(file) as f:
            data = json.load(f)
    except:
        number_count.append(0)
        continue
    l = []
    for item in data:
        
        try:
            sentence = list(item.values())[0]
    
            if ('$' in sentence or '%' in sentence):
                t = sentence.split()
                for word in t:
                    if '$' in word or '%' in sentence:
                        l.append(word)
        except:
            continue
    number_count.append(len(l))
    

In [30]:
feature_data['number_count'] = number_count

In [31]:
from transformers import AutoTokenizer, DistilBertForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained('has-abi/distilBERT-finetuned-resumes-sections')

model = DistilBertForSequenceClassification.from_pretrained("has-abi/distilBERT-finetuned-resumes-sections")

In [32]:
labels = ['awards','certificates','contact/name/title','education','interests','languages','para','professional_experiences','projects','skills','soft_skills','summary']

In [33]:
final_result = []
for file in file_path:
    d = {'awards':0,'certificates':0,'contact/name/title':0,'education':0,'interests':0,'languages':0,'para':0,'professional_experiences':0,'projects':0,'skills':0,'soft_skills':0,'summary':0}
    count = 0
    try:
        with open(file) as user_file:
            file_contents = user_file.read()
        list_file = eval(file_contents)
    
        for line in list_file:
            text = list(line.values())[0]
            encoded_input = tokenizer(text, return_tensors='pt')
            output = model(**encoded_input)
            label_id = torch.argmax(output.logits)
            label_id  = label_id.tolist()
            d[labels[label_id]] += 1
            count += 1


        for key in d:
            d[key]= d[key]/count
        final_result.append(d)
    except:
        final_result.append(d)
        continue

In [34]:
for i in labels:
    list1 = [j[i] for j in final_result]
    print(len(list1))
    if len(list1) == 130:
        feature_data[i] =list1

130
130
130
130
130
130
130
130
130
130
130
130


In [35]:
feature_data

,label,resume_name,sales_count,financial_count,investment_count,advisor_count,joy,anger,fear,number_count,...,contact/name/title,education,interests,languages,para,professional_experiences,projects,skills,soft_skills,summary
0,2,"21. Bocinsky, James_Resume.json",4,9,1,11,0.620690,0.172414,0.206897,72,...,8.000000,3.000000,3.000000,0.0,3.000000,7.000000,0.000000,0.000000,0.000000,12.000000
1,2,82. Linhardt Evan_Resume.json,0,0,0,0,0.769231,0.076923,0.153846,94,...,0.153846,0.153846,0.000000,0.0,0.153846,0.115385,0.000000,0.000000,0.000000,0.384615
2,2,"85. Malot, Dylan_Resume .PDF.json",0,2,1,3,0.769231,0.076923,0.153846,30,...,0.307692,0.076923,0.153846,0.0,0.038462,0.076923,0.000000,0.000000,0.000000,0.192308
3,2,"109. Poole, Blake - Resume.json",16,2,5,0,0.782609,0.086957,0.108696,49,...,7.000000,3.000000,0.000000,0.0,8.000000,14.000000,1.000000,0.000000,1.000000,8.000000
4,2,"136. Underkofler, Jon_Resume.json",10,10,8,7,0.780488,0.024390,0.195122,56,...,0.048780,0.073171,0.048780,0.0,0.146341,0.219512,0.000000,0.048780,0.024390,0.268293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1,"132. Statton, Robert H_BD.json",29,4,2,0,0.661017,0.118644,0.135593,49,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
126,1,"64. Kaempf, Andrea_Resume - FA - Seattle.json",18,0,7,3,0.610169,0.186441,0.169492,34,...,0.220339,0.050847,0.016949,0.0,0.084746,0.135593,0.000000,0.016949,0.000000,0.389831
127,1,"99. Moye, Jerry_Resume.json",64,0,0,0,0.594203,0.202899,0.202899,21,...,0.217391,0.028986,0.072464,0.0,0.101449,0.115942,0.000000,0.000000,0.014493,0.130435
128,1,"29. Corello, Daniel_Resume.json",63,6,1,6,0.604651,0.093023,0.302326,190,...,0.186047,0.046512,0.000000,0.0,0.023256,0.255814,0.000000,0.046512,0.000000,0.372093


In [36]:
l = []
final_r = []
for file in file_path:
    try:
        with open(file) as user_file:
            file_contents = user_file.read()
        list_file = eval(file_contents)
        for line in list_file:
            text = list(line.values())[0]
            results = readability.getmeasures(text, lang='en')
            score = results['readability grades']['FleschReadingEase']
            l.append(score)
        r = sum(l)/len(l)
        final_r.append(r)
            
    except:
        final_r.append(0)
        continue

In [37]:
feature_data['readability'] = final_r

In [38]:
feature_data

,label,resume_name,sales_count,financial_count,investment_count,advisor_count,joy,anger,fear,number_count,...,education,interests,languages,para,professional_experiences,projects,skills,soft_skills,summary,readability
0,2,"21. Bocinsky, James_Resume.json",4,9,1,11,0.620690,0.172414,0.206897,72,...,3.000000,3.000000,0.0,3.000000,7.000000,0.000000,0.000000,0.000000,12.000000,0.000000
1,2,82. Linhardt Evan_Resume.json,0,0,0,0,0.769231,0.076923,0.153846,94,...,0.153846,0.000000,0.0,0.153846,0.115385,0.000000,0.000000,0.000000,0.384615,53.482612
2,2,"85. Malot, Dylan_Resume .PDF.json",0,2,1,3,0.769231,0.076923,0.153846,30,...,0.076923,0.153846,0.0,0.038462,0.076923,0.000000,0.000000,0.000000,0.192308,48.626069
3,2,"109. Poole, Blake - Resume.json",16,2,5,0,0.782609,0.086957,0.108696,49,...,3.000000,0.000000,0.0,8.000000,14.000000,1.000000,0.000000,1.000000,8.000000,0.000000
4,2,"136. Underkofler, Jon_Resume.json",10,10,8,7,0.780488,0.024390,0.195122,56,...,0.073171,0.048780,0.0,0.146341,0.219512,0.000000,0.048780,0.024390,0.268293,34.981143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1,"132. Statton, Robert H_BD.json",29,4,2,0,0.661017,0.118644,0.135593,49,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
126,1,"64. Kaempf, Andrea_Resume - FA - Seattle.json",18,0,7,3,0.610169,0.186441,0.169492,34,...,0.050847,0.016949,0.0,0.084746,0.135593,0.000000,0.016949,0.000000,0.389831,0.000000
127,1,"99. Moye, Jerry_Resume.json",64,0,0,0,0.594203,0.202899,0.202899,21,...,0.028986,0.072464,0.0,0.101449,0.115942,0.000000,0.000000,0.014493,0.130435,0.000000
128,1,"29. Corello, Daniel_Resume.json",63,6,1,6,0.604651,0.093023,0.302326,190,...,0.046512,0.000000,0.0,0.023256,0.255814,0.000000,0.046512,0.000000,0.372093,50.930839


In [39]:
final_data = feature_data.sample(frac=1, random_state=42)

In [40]:
final_data

,label,resume_name,sales_count,financial_count,investment_count,advisor_count,joy,anger,fear,number_count,...,education,interests,languages,para,professional_experiences,projects,skills,soft_skills,summary,readability
55,0,"138. Vergari, Christine_Resume.json",22,2,0,4,0.694915,0.000000,0.305085,7,...,0.050847,0.033898,0.016949,0.000000,0.118644,0.0,0.135593,0.000000,0.423729,48.409532
40,0,"07. Arnold, Stephen_Resume.json",0,2,0,1,0.433333,0.433333,0.133333,0,...,0.166667,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.066667,0.133333,48.279288
19,2,"20. Birren, Nick_Resume.json",9,0,0,0,0.760000,0.160000,0.080000,136,...,0.080000,0.040000,0.040000,0.080000,0.240000,0.0,0.080000,0.040000,0.160000,46.625553
31,2,"83. Lopes, Michael_Resume.json",26,6,2,0,0.702703,0.162162,0.135135,21,...,0.081081,0.027027,0.000000,0.054054,0.108108,0.0,0.135135,0.027027,0.216216,45.469264
115,1,"58. Jacobs, Audrey_Resume.json",0,0,4,0,0.809524,0.047619,0.095238,44,...,0.190476,0.047619,0.000000,0.238095,0.095238,0.0,0.000000,0.000000,0.190476,50.439669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0,"91. Michie, Kirk_Resume_06.22.14.json",21,1,4,9,0.611111,0.148148,0.222222,96,...,0.074074,0.037037,0.000000,0.092593,0.166667,0.0,0.000000,0.000000,0.333333,47.880946
106,1,"79. Lenahan, John_CFP.json",8,10,11,16,0.459016,0.467213,0.057377,776,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
14,2,"137. Ventura, Nicholas_Resume_1_2019.json",2,3,4,0,0.678571,0.142857,0.178571,51,...,0.071429,0.107143,0.000000,0.000000,0.214286,0.0,0.035714,0.000000,0.321429,40.376452
92,0,"33. Danklef, Julia_Background.json",0,5,1,6,0.185185,0.388889,0.259259,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [41]:
final_data.to_csv('/home/guoy20/Data/feature_data.csv', index=False)